Tutorial 8: Model-Fit
=====================

You should now perform lens modeling using a pixelization, which is described fully in the example:

`autolens_workspace/*/imaging/features/pixelizations/modeling.py`

Once complete, you can then move on to the next HowToLens tutorial and gain more insight into adaptive
pixelizations.